# 2.6 MLflow in practice

# Scenario 3: Multiple data scientists working on multiple ML models

**MLflow setup**:
* **Tracking server**: yes, remote server (EC2)
* **Backend store**:   postgresql database
* **Artifact store**:  s3 bucket

The experiments can be explored by accessing the remote server. 

The exampe uses AWS to host a remote server. In order to run the example an AWS account is needed.   
`mlflow_on_aws.md` describes how to to create a new AWS account and launch the tracking server.

---

In [1]:
import mlflow
import os

# fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials
os.environ["AWS_PROFILE"] = "mlflow" 

# fill in with the public DNS of the EC2 instance
TRACKING_SERVER_HOST = "ec2-54-147-57-171.compute-1.amazonaws.com" 
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-147-57-171.compute-1.amazonaws.com:5000'


In [2]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-lg/1', creation_time=1748276468707, experiment_id='1', last_update_time=1748276468707, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-lg/0', creation_time=1748275019331, experiment_id='0', last_update_time=1748275019331, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/26 16:22:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-lg/1/081a0a1aa88840bcaa46ee31df44fc89/artifacts'
🏃 View run useful-goose-487 at: http://ec2-54-147-57-171.compute-1.amazonaws.com:5000/#/experiments/1/runs/081a0a1aa88840bcaa46ee31df44fc89
🧪 View experiment at: http://ec2-54-147-57-171.compute-1.amazonaws.com:5000/#/experiments/1


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-lg/1', creation_time=1748276468707, experiment_id='1', last_update_time=1748276468707, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-lg/0', creation_time=1748275019331, experiment_id='0', last_update_time=1748275019331, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [5]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
client.search_registered_models()

[]

In [9]:
runs = client.search_runs(
    experiment_ids=['1'],
    order_by=['start_time DESC'],
    max_results=1
)

if runs:
    run_id = runs[0].info.run_id
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name='iris-classifier'
    )
else:
    print("No runs found in experiment 1")

Successfully registered model 'iris-classifier'.
2025/05/26 16:27:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.
